In [ ]:
from google.colab import drive
drive.mount('/content/drive')

pathG='/content/drive/MyDrive/Pract/data' # @Reece, change this to your PathG
# @Reece, put greyimg.zip in your pathG

#pathG='../data'



#cp = '/content/drive/MyDrive/Pretrained/wav2vec_small.pt'
#sample="/content/drive/MyDrive/Pract/data/voice_samples/1173_GM1001_1326493712.wav"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun  7 20:48:17 2022

@author: sanjeev
"""

import argparse
import math
import sys
import time
import copy
import numpy as np

import keras
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization #, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical


import os

#os.chdir('/media/sanjeev/Data/Pract/Practicum/codesNdata/mycode')
#pathG='../data'

import pandas as pd
df=pd.read_csv(pathG+"/labels/Yared Alemu_fear.csv")
df = df.reset_index()
df['name2'] =df['name'].apply (lambda x: x.split(".")[0]+".png")


import glob
from PIL import Image
import os
imgPath= pathG+"/greyimg" 

import cv2
imgPath = "/content/drive/MyDrive/Pract/data/images"

resize=224


filenames= glob.glob(imgPath+"/*.png" )
filenames = [os.path.basename(x) for x in filenames]

fl= imgPath+"/"+df.iloc[0]['name2']
img = cv2.imread(fl)
res = cv2.resize(img, dsize=(resize, resize), interpolation=cv2.INTER_CUBIC)
x = (res/resize).reshape(1,resize,resize,3)
arr = x #np.expand_dims(x, axis=0) 
#print(arr.shape)
for i,row in df.iterrows():
    if i>0:
        fl = imgPath+"/"+row['name2']
        img = cv2.imread(fl)
        res = cv2.resize(img, dsize=(resize, resize), interpolation=cv2.INTER_CUBIC)
        x = (res/resize) .reshape(1,resize,resize,3)  
        
        arr2 = x #np.expand_dims(ft, axis=0)
        #print(arr2.shape)
        arr = np.vstack((arr,arr2))
labels=np.array(df.emotion)


In [ ]:
arr.shape

(530, 224, 224, 3)

In [ ]:

from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(arr, labels, test_size = 0.3)


# converting training images into torch format
#train_x = train_x.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2],1)


# converting the target into torch format
train_y = train_y.astype(int)


# shape of training data
train_x.shape, train_y.shape
# converting validation images into torch format
#val_x = val_x.reshape(val_x.shape[0],  val_x.shape[1], val_x.shape[2],1)


# converting the target into torch format
val_y = val_y.astype(int)




In [ ]:


#K.set_image_dim_ordering('th')
#print(K.image_data_format())

## required for efficient GPU use
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#from tensorflow.keras.backend import tensorflow_backend
from tensorflow.compat.v1.keras.backend import set_session
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
set_session(session)
## required for efficient GPU use
## required for efficient GPU use

import os

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

# define path to save model
model_path = './fm_cnn_BN.h5'
# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]


# get data


#reshape data
y_train_CNN = train_y
X_train_CNN = train_x
print('train shape after reshape: {}'.format(X_train_CNN.shape))

y_test_CNN = val_y
X_test_CNN = val_x
print('test shape after reshape: {}'.format(X_test_CNN.shape))

# one hot encode outputs
y_train_CNN = to_categorical(y_train_CNN)
y_test_CNN = to_categorical(y_test_CNN)
num_classes = y_train_CNN.shape[1]

# normalize inputs from 0-255 to 0-1
#X_train_CNN = X_train_CNN / 255
#X_test_CNN = X_test_CNN / 255

#size of parameters
batch_size = 32
num_classes = 2
epochs = 50
filter_pixel=3
noise = 1
droprate=0.25

# input image dimensions
img_rows, img_cols = 224, 224

input_shape = ( img_rows, img_cols,3)

#Start Neural Network
model = Sequential()
#convolution 1st layer
model.add(Conv2D(8, kernel_size=(3, 3), 
                 activation='relu', padding="same",
                 input_shape=input_shape)) 
model.add(AveragePooling2D(pool_size=(2, 2)))


#convolution 2nd layer
model.add(Conv2D(16, kernel_size=(3, 3),  padding="same",
                 activation='relu')) 
model.add(AveragePooling2D(pool_size=(2, 2)))

#convolution 3rd layer
model.add(Conv2D(32, kernel_size=(3, 3),  padding="same",
                 activation='relu')) 
model.add(AveragePooling2D(pool_size=(2, 2)))


#Fully connected 1st layer
model.add(Flatten())
model.add(Dense(25088,use_bias=True)) 
model.add(Activation('relu')) 
#model.add(Dropout(droprate)) 

model.add(Dense(2048,use_bias=True)) 
model.add(Activation('relu'))   


model.add(Dense(2048, use_bias=True)) 
model.add(Activation('relu'))      

#Fully connected final layer
model.add(Dense(2)) 
model.add(Activation('softmax')) 



train shape after reshape: (371, 224, 224, 3)
test shape after reshape: (159, 224, 224, 3)


In [ ]:

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.rmsprop_v2.RMSProp(),
              metrics=[2*tf.keras.metrics.Recall() * tf.keras.metrics.Precision()/(tf.keras.metrics.Recall()+tf.keras.metrics.Precision())])

model.summary()

#Save Model=ON
history = model.fit(X_train_CNN, y_train_CNN,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test_CNN, y_test_CNN),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test_CNN, y_test_CNN, verbose=1)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

TypeError: ignored